In [11]:
import re
import pandas as pd
import numpy as np
import os
import requests
import src.utils as u
from src.exceptions import UrlNotFoundError
from bs4 import BeautifulSoup
from dotenv import load_dotenv

In [6]:
oscars = pd.read_csv("./INPUT/the_oscar_award.csv")
pd.set_option('display.max_rows', 6000)
pd.set_option('display.max_columns', 500)
oscars.head()

,year_film,year_ceremony,ceremony,category,name,film,win
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


# Para API

In [7]:
load_dotenv()
token = os.getenv("API_KEY")

In [ ]:
def requestGithub(endpoint):
    token = os.getenv("GITHUB_APIKEY")
    if not token:
        raise ValueError("You must set a GITHUB_APIKEY token")
    
    baseUrl = "https://api.github.com"
    url = baseUrl+endpoint
    print(f"Requesting data from {url}")
    headers = {
        "Authorization": f"token {token}"
    }
    res = requests.get(url,headers=headers)
    if res.status_code != 200:
        print(res.text)
        raise ValueError("Bad Response")
    return res.json()

In [ ]:
requestGithub("/user/emails",token)
# Url with url encoded query params (?key1=value&key2=value)
# https://www.urlencoder.org/
data = requestGithub("/repos/ironhack-datalabs/datamad0120/pulls?state=open&sort=updated")

# Para Scrapping

In [18]:
def find_url(film):
    url = 'https://www.google.com/search?q=' + film.replace(" ", "+") + "+filmaffinity"
    URL_FILMS = "https://www.filmaffinity.com/es/film"
    search = f'?q={film} filmaffinity'
    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')
    tag = [e.attrs["href"] for e in soup.find_all("a") if "href" in e.attrs and URL_FILMS in e.attrs["href"]][0]
    return (re.search(URL_FILMS +"\d+",tag)).group()+".html"

In [22]:
a = find_url("los lunes")
print(a)

https://www.filmaffinity.com/es/film487418.html


In [16]:
def find_movie_rate(film):
    data = requests.get(find_url(film)).text
    soup = BeautifulSoup(data, 'html.parser')
    return float(soup.select_one('div[id^="movie-rat-avg"]').text.strip().replace(",", "."))

In [21]:
find_movie_rate("los lunes")

7.4